In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.4.0`

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[4]")
      .getOrCreate()

import spark.implicits._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

Logger.getRootLogger().setLevel(Level.ERROR)

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [ ]:
import org.apache.spark.sql.expressions.Window

val data = spark.read.parquet("../data/monthParquet/").na.drop()

val window = Window.partitionBy("mes").orderBy($"ta_max".desc)

val dataWindow = data
    .withColumn("ta_max", func.split($"ta_max", "\\(")(0).cast(DoubleType))
    .groupBy($"fecha")
    .agg(func.avg($"ta_max").alias("ta_max"))
    .select(func.month($"fecha").alias("mes"), func.year($"fecha").alias("año"), $"ta_max")
    .withColumn("dense_rank", func.dense_rank().over(window))
    .filter($"dense_rank" === 1)

In [ ]:
def countByYear(year : Int) : Long = {
    dataWindow
        .filter($"año" === year)
        .count()
}
 
val year2010 = countByYear(2010)
val year2011 = countByYear(2011)
val year2012 = countByYear(2012)
val year2013 = countByYear(2013)
val year2014 = countByYear(2014)
val year2015 = countByYear(2015)
val year2016 = countByYear(2016)
val year2017 = countByYear(2017)
val year2018 = countByYear(2018)
val year2019 = countByYear(2019)
val year2020 = countByYear(2020)
val year2021 = countByYear(2021)
val year2022 = countByYear(2022)

In [ ]:
dataWindow
    .withColumn("temperatura maxima", func.round($"ta_max", 2))
    .orderBy($"mes".asc)
    .select($"mes", $"año", $"temperatura maxima")
    .show()

val dataToPlot = Seq(
  Bar(
    Seq(2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022),
    Seq(year2010, year2011, year2012, year2013, year2014, year2015, year2016, 
        year2017, year2018, year2019, year2020, year2021, year2022),
    marker = Marker(
        color = Color.RGB(255, 174, 0),
        opacity = 0.6,
        line = Line(
            color = Color.RGB(189, 129, 0),
              width = 1.5
        )
    )
  )
)

plot(dataToPlot)